In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [15]:
# Read in Loans dataset
# Set display to show all columns

pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/SBAnational.csv")
df

c:\users\user\.virtualenvs\build_week_2-2dp-f4_g\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,9995573004,FABRIC FARMS,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,27-Feb-97,1997,60,6,1.0,0,0,1,0,0,N,NaN,30-Sep-97,"$70,000.00",$0.00,P I F,$0.00,"$70,000.00","$56,000.00"
899160,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,27-Feb-97,1997,60,6,1.0,0,0,1,0,Y,N,NaN,31-Oct-97,"$85,000.00",$0.00,P I F,$0.00,"$85,000.00","$42,500.00"
899161,9995613003,"RADCO MANUFACTURING CO.,INC.",SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,27-Feb-97,1997,108,26,1.0,0,0,1,0,N,N,NaN,30-Sep-97,"$300,000.00",$0.00,P I F,$0.00,"$300,000.00","$225,000.00"
899162,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,27-Feb-97,1997,60,6,1.0,0,0,1,0,N,Y,8-Mar-00,31-Mar-97,"$75,000.00",$0.00,CHGOFF,"$46,383.00","$75,000.00","$60,000.00"


I can see there are some zeros under NAICS so I have to check for null values before wrangling the zeros as NaNs

In [16]:
df.isnull().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

The *ChgOffDate* column is the date that the loan gets "charged off" to the SBA (Defaulting). I'll drop this column to prevent leakage

In [18]:
df = df.drop("ChgOffDate", axis=1)

The first idea for feature engineering I had was to check if the borrower's state is the same as the bank's. I'll have to drop those rows of NaNs in *BankState* to make that possible. 1,566 rows looks like a lot but I can live with that since I have 899,000 observations.

In [19]:
# Drop Bank state null rows
df = df[df["BankState"].notna()]

*MIS_Status* is the target variable so I have to drop the rows that are null as well

In [20]:
# Drop target variable with no values

df = df[df["MIS_Status"].notna()]

In [21]:
# Check number of observations and update NaN count

print(len(df))
df.isnull().sum()

895654


LoanNr_ChkDgt           0
Name                   14
City                   28
State                  13
Zip                     0
Bank                    0
BankState               0
NAICS                   0
ApprovalDate            0
ApprovalFY              0
Term                    0
NoEmp                   0
NewExist              134
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4511
LowDoc               2578
DisbursementDate     2173
DisbursementGross       0
BalanceGross            0
MIS_Status              0
ChgOffPrinGr            0
GrAppv                  0
SBA_Appv                0
dtype: int64

In [22]:
df[df["City"].isnull()]

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
42535,1330603010,BUSATH PHOTOGRAPHY,NaN,UT,84109,MOUNTAIN W. SMALL BUS. FINAN,UT,0,15-Dec-81,1982,300,2,2.0,0,0,0,0,N,N,6-Jan-82,"$190,000.00",$0.00,P I F,$0.00,"$190,000.00","$190,000.00"
177891,2268732010,"HAYES/DOCKSIDE,INC",NaN,LA,70130,NEW ORLEANS REGIONAL BUS. DEVE,LA,0,17-Dec-81,1982,240,23,2.0,0,0,0,0,N,N,10-Feb-82,"$500,000.00",$0.00,P I F,$0.00,"$500,000.00","$500,000.00"
243450,2716822007,"YACHTING &NAVIGATION, LTD",NaN,IL,60624,BANK - AMERICA NATL ASSOC,IL,0,10-Apr-81,1981,300,10,2.0,0,0,0,0,N,N,4-Nov-81,"$150,000.00",$0.00,P I F,$0.00,"$150,000.00","$150,000.00"
270263,2884182004,"HERMANOS LOPEZ,INC. ""B""",NaN,CA,92102,CDC SMALL BUS. FINAN CORP,CA,0,16-Apr-81,1981,180,45,2.0,0,0,0,0,N,N,6-Jan-82,"$488,000.00",$0.00,P I F,$0.00,"$488,000.00","$488,000.00"
270265,2884192007,"RODIECK PLUMBING SUPPLY,INC ""A",NaN,CA,92102,CDC SMALL BUS. FINAN CORP,CA,0,15-Jun-81,1981,300,20,2.0,0,0,0,0,N,N,10-Sep-81,"$203,000.00",$0.00,P I F,$0.00,"$211,000.00","$211,000.00"
270268,2884202008,"SAN DIEGO REFRIGERATD SVCS,INC",NaN,CA,92102,CDC SMALL BUS. FINAN CORP,CA,0,26-Jun-81,1981,180,28,1.0,0,0,0,0,N,N,10-Sep-81,"$407,000.00",$0.00,P I F,$0.00,"$407,000.00","$407,000.00"
270270,2884212000,"EMACO, INC. ""B""",NaN,CA,92102,CDC SMALL BUS. FINAN CORP,CA,0,13-Jul-81,1981,180,7,2.0,0,0,0,0,N,N,10-Sep-81,"$105,000.00",$0.00,P I F,$0.00,"$105,000.00","$105,000.00"
270277,2884232006,"MAYER BLUEPRINT,INC. ""B""",NaN,CA,92102,CDC SMALL BUS. FINAN CORP,CA,0,29-Jul-81,1981,240,20,2.0,0,0,0,0,N,N,10-Sep-81,"$159,000.00",$0.00,P I F,$0.00,"$159,000.00","$159,000.00"
270284,2884252001,TEX-WIPE COMPANY,NaN,CA,92102,CDC SMALL BUS. FINAN CORP,CA,0,31-Aug-81,1981,240,26,2.0,0,0,0,0,N,N,4-Nov-81,"$365,000.00",$0.00,P I F,$0.00,"$365,000.00","$365,000.00"
326169,3258222000,LAKEWOOD HEALTH CARE CENTER LT,NaN,OH,44115,GROWTH CAPITAL CORP.,OH,0,18-Nov-81,1982,300,40,1.0,0,0,0,0,N,N,10-Feb-82,"$205,000.00",$0.00,P I F,$0.00,"$205,000.00","$205,000.00"


I was hoping the State null rows were in the same as the City null rows so I could find them at once. Looks like I'll have to research each column one at a time from the zipcode

I stumbled upon a brilliant package that can update City and State from zip codes. Check out [*uszipcode*](https://pypi.org/project/uszipcode/0.1.2/)

In [67]:
# I tried ZipcodeSearchEngine like in the documentation but got an import error
# Autofill had a SearchEngine class so try that
from uszipcode import SearchEngine
search = SearchEngine()

In [75]:
# Assign null States df subset to a smaller df
df_State_null = df[df["State"].isnull()].copy(deep = True)

In [79]:
df_State_null

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
49244,1380800010,TRYON COATS & LEATHER,JOHNSTOWN NY,NaN,0,KEYBANK NATIONAL ASSOCIATION,NY,0,18-May-66,1966,282,0,0.0,0,0,0,0,N,N,16-Aug-66,"$60,000.00",$0.00,CHGOFF,"$6,084.00","$60,000.00","$54,000.00"
264664,2850643009,CENTURY 21 PHILLIPS REALTY,CAMERON PARK,NaN,95682,SBA - EDF ENFORCEMENT ACTION,CO,0,12-Feb-87,1987,240,19,1.0,0,19,0,0,N,N,17-Jun-87,"$81,000.00",$0.00,P I F,$0.00,"$83,000.00","$83,000.00"
306274,3113583009,THE COMPUTER EDGE,"BOX 267, APO AP",NaN,96205,RTC/WESTPORT SAVINGS BANK,CA,541511,21-Jan-88,1988,73,8,1.0,0,0,0,0,N,N,16-Mar-88,"$50,000.00",$0.00,CHGOFF,"$30,589.00","$50,000.00","$45,000.00"
328526,3280213000,KING'S TRUCK WASH,WICHITA,NaN,67219,SOUTH CENT. KANSAS ECONOMIC DE,KS,811192,5-Aug-88,1988,120,17,2.0,10,7,0,0,N,N,12-Jul-89,"$119,000.00",$0.00,P I F,$0.00,"$119,000.00","$119,000.00"
351072,3445553007,JAMES A. NICHOLS,"1542,TABUK,SAUDI ARABIA",NaN,79925,WELLS FARGO BANK NATL ASSOC,TX,532230,19-May-89,1989,16,1,2.0,0,0,0,0,N,N,31-Jul-89,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$54,000.00"
366139,3556993002,"GOLF SHOES UNLIMITED, INC.",PALM BEACH GARDENS,NaN,33410,"BEAL BANK, SSB",TX,451110,19-Oct-89,1990,84,3,2.0,0,0,1,0,N,N,31-Jan-90,"$75,000.00",$0.00,P I F,$0.00,"$75,000.00","$67,500.00"
366158,3557093005,CASCO SERVICE,CASCO,NaN,54205,UNION STATE BANK,WI,0,19-Oct-89,1990,204,8,1.0,0,0,1,0,N,N,30-Apr-90,"$300,000.00",$0.00,P I F,$0.00,"$300,000.00","$252,600.00"
367007,3563473008,P & P TOOL INC,SOMERSET,NaN,54025,FIRST BANK OF BALDWIN,WI,0,26-Oct-89,1990,240,7,1.0,0,0,1,0,N,N,31-Jan-90,"$82,000.00",$0.00,P I F,$0.00,"$82,000.00","$73,800.00"
379174,3664443009,WILLIAMS JEWELERS,SALT LAKE CITY,NaN,84124,ZIONS FIRST NATIONAL BANK,UT,448310,14-Dec-89,1990,60,4,1.0,0,0,1,0,N,N,31-Jul-90,"$250,000.00",$0.00,P I F,$0.00,"$250,000.00","$212,500.00"
385418,3714383003,"LAKE MOVIE TYMES, INC.",LAKE OZARK,NaN,65049,CENTRAL BK OF LAKE OF OZARKS,MO,532230,4-Jan-90,1990,60,1,1.0,0,0,1,0,N,N,31-Jan-90,"$75,000.00",$0.00,P I F,$0.00,"$75,000.00","$67,500.00"


I can see from the States subset that in addition to high cardinality, there also wrong values in *City*. Can only compare States anyway so drop that column

In [80]:
# Drop City column
df = df.drop("City", axis=1)

In [81]:
# Create functions to use zipcode for state

def state_finder(x):
    zipcode = search.by_zipcode(x)
    state = zipcode.state
    return state if state else np.NaN

In [82]:
for i in df_State_null.index:
    df_State_null.loc[i, "State"] = state_finder(df_State_null.loc[i, "Zip"])

In [83]:
df_State_null

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
49244,1380800010,TRYON COATS & LEATHER,JOHNSTOWN NY,NaN,0,KEYBANK NATIONAL ASSOCIATION,NY,0,18-May-66,1966,282,0,0.0,0,0,0,0,N,N,16-Aug-66,"$60,000.00",$0.00,CHGOFF,"$6,084.00","$60,000.00","$54,000.00"
264664,2850643009,CENTURY 21 PHILLIPS REALTY,CAMERON PARK,CA,95682,SBA - EDF ENFORCEMENT ACTION,CO,0,12-Feb-87,1987,240,19,1.0,0,19,0,0,N,N,17-Jun-87,"$81,000.00",$0.00,P I F,$0.00,"$83,000.00","$83,000.00"
306274,3113583009,THE COMPUTER EDGE,"BOX 267, APO AP",NaN,96205,RTC/WESTPORT SAVINGS BANK,CA,541511,21-Jan-88,1988,73,8,1.0,0,0,0,0,N,N,16-Mar-88,"$50,000.00",$0.00,CHGOFF,"$30,589.00","$50,000.00","$45,000.00"
328526,3280213000,KING'S TRUCK WASH,WICHITA,KS,67219,SOUTH CENT. KANSAS ECONOMIC DE,KS,811192,5-Aug-88,1988,120,17,2.0,10,7,0,0,N,N,12-Jul-89,"$119,000.00",$0.00,P I F,$0.00,"$119,000.00","$119,000.00"
351072,3445553007,JAMES A. NICHOLS,"1542,TABUK,SAUDI ARABIA",TX,79925,WELLS FARGO BANK NATL ASSOC,TX,532230,19-May-89,1989,16,1,2.0,0,0,0,0,N,N,31-Jul-89,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$54,000.00"
366139,3556993002,"GOLF SHOES UNLIMITED, INC.",PALM BEACH GARDENS,FL,33410,"BEAL BANK, SSB",TX,451110,19-Oct-89,1990,84,3,2.0,0,0,1,0,N,N,31-Jan-90,"$75,000.00",$0.00,P I F,$0.00,"$75,000.00","$67,500.00"
366158,3557093005,CASCO SERVICE,CASCO,WI,54205,UNION STATE BANK,WI,0,19-Oct-89,1990,204,8,1.0,0,0,1,0,N,N,30-Apr-90,"$300,000.00",$0.00,P I F,$0.00,"$300,000.00","$252,600.00"
367007,3563473008,P & P TOOL INC,SOMERSET,WI,54025,FIRST BANK OF BALDWIN,WI,0,26-Oct-89,1990,240,7,1.0,0,0,1,0,N,N,31-Jan-90,"$82,000.00",$0.00,P I F,$0.00,"$82,000.00","$73,800.00"
379174,3664443009,WILLIAMS JEWELERS,SALT LAKE CITY,UT,84124,ZIONS FIRST NATIONAL BANK,UT,448310,14-Dec-89,1990,60,4,1.0,0,0,1,0,N,N,31-Jul-90,"$250,000.00",$0.00,P I F,$0.00,"$250,000.00","$212,500.00"
385418,3714383003,"LAKE MOVIE TYMES, INC.",LAKE OZARK,MO,65049,CENTRAL BK OF LAKE OF OZARKS,MO,532230,4-Jan-90,1990,60,1,1.0,0,0,1,0,N,N,31-Jan-90,"$75,000.00",$0.00,P I F,$0.00,"$75,000.00","$67,500.00"


In [84]:
# Update dataframe
df.update(df_State_null)

In [85]:
df.isnull().sum()

LoanNr_ChkDgt           0
Name                   14
State                   2
Zip                     0
Bank                    0
BankState               0
NAICS                   0
ApprovalDate            0
ApprovalFY              0
Term                    0
NoEmp                   0
NewExist              134
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4511
LowDoc               2578
DisbursementDate     2173
DisbursementGross       0
BalanceGross            0
MIS_Status              0
ChgOffPrinGr            0
GrAppv                  0
SBA_Appv                0
dtype: int64

In [86]:
# Look at two rows with null States

df[df["State"].isnull()]

,LoanNr_ChkDgt,Name,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
49244,1.380800e+09,TRYON COATS & LEATHER,NaN,0.0,KEYBANK NATIONAL ASSOCIATION,NY,0.0,18-May-66,1966,282.0,0.0,0.0,0.0,0.0,0.0,0.0,N,N,16-Aug-66,"$60,000.00",$0.00,CHGOFF,"$6,084.00","$60,000.00","$54,000.00"
306274,3.113583e+09,THE COMPUTER EDGE,NaN,96205.0,RTC/WESTPORT SAVINGS BANK,CA,541511.0,21-Jan-88,1988,73.0,8.0,1.0,0.0,0.0,0.0,0.0,N,N,16-Mar-88,"$50,000.00",$0.00,CHGOFF,"$30,589.00","$50,000.00","$45,000.00"


In [87]:
# Drop both rows
df = df.drop([49244, 306274])

In [89]:
# Don't need the Name of the business either so drop that column

df = df.drop("Name", axis=1)

In [90]:
# That concludes cleaning of loanee data
# Check size of data frame
df.shape

(895652, 24)

In [95]:
# With over 895,000 observations, I'm good with dropping the remaining NaN rows
# Quick check first to confirm null columns and amount

df.isnull().sum()

LoanNr_ChkDgt           0
State                   0
Zip                     0
Bank                    0
BankState               0
NAICS                   0
ApprovalDate            0
ApprovalFY              0
Term                    0
NoEmp                   0
NewExist              134
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4511
LowDoc               2578
DisbursementDate     2173
DisbursementGross       0
BalanceGross            0
MIS_Status              0
ChgOffPrinGr            0
GrAppv                  0
SBA_Appv                0
dtype: int64

In [96]:
# I'm going against imputing these values since the dataset is quite large
# but also because I think these columns could really affect my model

df = df.dropna()

In [97]:
# Another look at size and null count
print(df.shape)
df.isnull().sum()

(886293, 24)


LoanNr_ChkDgt        0
State                0
Zip                  0
Bank                 0
BankState            0
NAICS                0
ApprovalDate         0
ApprovalFY           0
Term                 0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementDate     0
DisbursementGross    0
BalanceGross         0
MIS_Status           0
ChgOffPrinGr         0
GrAppv               0
SBA_Appv             0
dtype: int64

In [91]:
# Want to look at range of Fiscal year the loan was approved in ApprovalFY
df["ApprovalFY"].value_counts(dropna=False).index

Index([   2006,    2007,    2003,    2005,  '1995',    2002,  '2004',    2008,
          1997,    2000,    1999,    2001,    1996,    1998,  '2005',    2004,
          1993,    1992,    1994,    2009,    2010,    1991,    1990,    1989,
          2011,  '1994',    2012,  '1996',    2013,    1987,    1986,    1984,
          1985,    1988,    1983,    1982,    1981,    1980,    1979,    2014,
          1978,  '1977',  '1978',  '1979',  '1976',    1973,    1974,    1972,
        '1981', '1976A',    1975,  '1980',  '1982',  '1971',  '1975',    1977,
          1976,  '1970',  '1983',  '1984',  '1974',    1971,  '1969',  '1968'],
      dtype='object')

In [92]:
# Correct 1976A error
df["ApprovalFY"] = df["ApprovalFY"].replace({"1976A" : 1976})

In [93]:
# Change column to integers
df.ApprovalFY = df.ApprovalFY.astype(int)

In [94]:
df.ApprovalFY.value_counts().sort_index()

1968        1
1969        3
1970        8
1971       18
1972       25
1973       48
1974       40
1975       29
1976       81
1977      134
1978      235
1979      344
1980      440
1981      588
1982      702
1983     1640
1984     1982
1985     1917
1986     2100
1987     2206
1988     1892
1989    12974
1990    14646
1991    15455
1992    20687
1993    23152
1994    31472
1995    45577
1996    39980
1997    37711
1998    35999
1999    37347
2000    37352
2001    37317
2002    44307
2003    58000
2004    68195
2005    76952
2006    75751
2007    71649
2008    39457
2009    19103
2010    16828
2011    12593
2012     5992
2013     2455
2014      268
Name: ApprovalFY, dtype: int64

Dropping *ChgOffPrinGr* column as well to prevent more leakage

In [98]:
df = df.drop("ChgOffPrinGr", axis=1)

In [100]:
# Look at the dtypes
df.dtypes

LoanNr_ChkDgt        float64
State                 object
Zip                  float64
Bank                  object
BankState             object
NAICS                float64
ApprovalDate          object
ApprovalFY             int32
Term                 float64
NoEmp                float64
NewExist             float64
CreateJob            float64
RetainedJob          float64
FranchiseCode        float64
UrbanRural           float64
RevLineCr             object
LowDoc                object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
GrAppv                object
SBA_Appv              object
dtype: object

Dont need *LoanNr_ChkDgt*, *Zip* should be an object, dates should be changed to datetime objects, while the last object columns should be changed to integers, assuming loans only give out full dollar amounts

In [101]:
# Drop id type column
df = df.drop("LoanNr_ChkDgt", axis = 1)

# Convert Zip column
df["Zip"] = df["Zip"].astype(objectt)

In [102]:
# Convert approval date and disbursement date

df["ApprovalDate"] = pd.to_datetime(df["ApprovalDate"], infer_datetime_format= True)
df["DisbursementDate"] = pd.to_datetime(df["DisbursementDate"], infer_datetime_format= True)

In [116]:
# Look at one of the $ columns to figure how to clean
df.loc[0, "DisbursementGross"]

'$60,000.00 '

Have to remove the $ sign, the comma, the dot, decimals and space at the end

In [108]:
def string_cleaner(amount):
    amount = amount[:-4].strip("$").replace(",", "")
    return int(amount)

In [114]:
# Create subset list to be cleaned
ending = ("Gross", "Appv")
string_columns = [col for col in df if col.endswith(ending)]
string_columns

['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']

In [119]:
for col in string_columns:
    df[col] = df[col].apply(string_cleaner)

In [120]:
# Look at dtypes again
df.dtypes

State                        object
Zip                          object
Bank                         object
BankState                    object
NAICS                       float64
ApprovalDate         datetime64[ns]
ApprovalFY                    int32
Term                        float64
NoEmp                       float64
NewExist                    float64
CreateJob                   float64
RetainedJob                 float64
FranchiseCode               float64
UrbanRural                  float64
RevLineCr                    object
LowDoc                       object
DisbursementDate     datetime64[ns]
DisbursementGross             int64
BalanceGross                  int64
MIS_Status                   object
GrAppv                        int64
SBA_Appv                      int64
dtype: object